## CART Algorithms

- Classification
    - Decision Tree Classification
    - Random Forest Classification
    - Bagging Classification

- Regression
    - Decision Tree Regression
    - Random Forest Regression
    - Bagging Regression

- Ensemble Models
    - Bagging
    - Boosting
        - ADA Boosting (Adaptive Boosting)
        - Gradient Boosting (GB Boosting)
        - XG Boosting  (Extreme Gradient)
    - Stacking
        - Blended Stacking Bootstrap Aggregation

### Classification

CART - Classification And Regression Trees

Used for non linear data. 

Decision trees are the basis for the classification algorithm.

#### Decision Tree Classification

Binary classification problem - When your y variable has only 2 classes.
Multi classification problem - When your y variable has more than 2 classes.

Consider the below dataset, where we are trying to predict a persons EMI repayment capability based on his gender, existing car loan,existing home loan
<img src='img/dt-2.png'/>

When can try to draw a tree as similar to the one in the right. let's try to interpret the tree.

If we consider Gender as the root, 
- we have total 12 people 
    - out of which five people are in '0' category
    - seven people are in '1' category. 

Considering second level as House Loan, 
- out of the five people who belongs to category '0' gender
    - one doesnt have housing loan
    - four has housing loan
    
- out of the seven people who belongs to category '1' gender
    - four doesnt have housing loan
    - three has housing loan
    
and so on .. I believe you got the drift

The maximum level of a tree, will be the number of features available in the dataset. <b> A fully grown tree </b> is a tree with all the columns considered.

Once we have constructed a tree, let's consider a new row for which we have to predict the y value. Take each attribute of the new row and see which path will the new row take to reach a decision. Once you get to the leaf node you can say out of all the people with similar characteristics, 'm' is the probability that he has the capacity to pay the EMI and 'n' is the probability he doesnt not  have the capacity to pay the EMI. i.e. Majority voting applies. In regression problems, we take the mean of the people available.

With fully grown decision trees, one of the major problem is 'Overfitting'. i.e. it works well on test data , but not in train data. i.e. Decision Tree is harder to generalize.

#### Pruning
How do we control/restrict the growth of a tree ?  Answer is <b> Pruning </b>

How to Prune ?
- Instead of considering all columns, we can pick important columns that's one way to Prune the data
- On the decision nodes(nodes are called decision nodes), we can put conditions. i.e. create the node only if there are more than 'n' number of observations.

#### How to decide which variables to be considered on what level of the tree ?
Imagine a dataset where we have the following, 
- people who likes watching  movies .. let's imagine the distribution as 60 : 40
- people who likes watching games .. let's imagine the distribition as 70 : 30 

Which one would you consider for the root node ? Ofcourse people who likes watching movies, because the tree will be balanced better on both sides with that choice. 

The best split will be when , 
within the split, data has similar characteristics. 
Across splits, data should have different characteristics.

To measure similarity we have 4 ways
<b>
<pre>
1) Entropy
2) Information Gain
3) GINI Index
4) CHAID
</pre>
</b>

With these 4 ways , we can find the perfect column to start the tree.

i.e. let's imagine x1,x2,x3,x4 and y as the dataset. We find information gain for each variables w.r.t to y

find information gain of x1,y
information gain of x2,y
information gain of x3,y
information gain of x4,y

The field with the maximum information gain can be chosen as the first node

##### Entropy Calculation

Entropy is often interpreted as the degree of disorder or randomness in the system. Entropy will be a value between 0 and 1. 

0 - means it's a homogeneous mixture

1 - means it's a heterogeneous mixture

Read the article <a href='https://www.saedsayad.com/decision_tree.htm'>here</a>

a) Entropy using the frequency table of one attribute:

<img src='img/entropy-1.png'/>

Probability of playing golf = 9/14 = 0.64

Probability of not playing golf = 5/14 = 0.36

This is only with respect to one variable, we can also calculate entropy for x1,y | x2,y and so on

b) Entropy using the frequency table of two attributes:

<img src='img/entropy-2.png'/>

##### Program for Entropy calculation

First let's calculate entropy for y variable alone (i.e.  "Entropy using the frequency table of one attribute" - formulae 1  in the above)

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [4]:
dataset = df[['survived','sex','who','adult_male','fare']]
dataset

,survived,sex,who,adult_male,fare
0,0,male,man,True,7.2500
1,1,female,woman,False,71.2833
2,1,female,woman,False,7.9250
3,1,female,woman,False,53.1000
4,0,male,man,True,8.0500
...,...,...,...,...,...
886,0,male,man,True,13.0000
887,1,female,woman,False,30.0000
888,0,female,woman,False,23.4500
889,1,male,man,True,30.0000


In [7]:
# Even we can use continous variables, but we convert them to binary
x = []
for i in dataset['fare']:
    if i > dataset['fare'].mean() :
        x.append(1)
    else:
        x.append(0)
dataset['fare'] = x
dataset.isna().sum()

C:\Users\Rani\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


survived      0
sex           0
who           0
adult_male    0
fare          0
dtype: int64

In [8]:
# we can consider y variable as survived
y = dataset['survived']
x = dataset.drop('survived',axis=1)

y.shape, x.shape

((891,), (891, 4))

In [10]:
y.value_counts()

0    549
1    342
Name: survived, dtype: int64

In [12]:
probabilites = y.value_counts().values/y.value_counts().sum()
p1 = probabilites[0]
p2 = probabilites[1]
p1,p2

(0.6161616161616161, 0.3838383838383838)

In [13]:
import math
-p1 * math.log(p1,2)-p2 * math.log(p2,2)

# Entropy for y = 0.9607079018756469

0.9607079018756469

<b> Now let's calculate Entropy for E(T,X) i.e. target variable and one of the X variable. Let's consider survived and gender </b>

In [15]:
#E(Survived,Gender)
entropy_x1 = pd.crosstab(df['survived'],df['sex']).T
entropy_x1

survived,0,1
sex,,
female,81,233
male,468,109


In [17]:
entropy_x1.values

array([[ 81, 233],
       [468, 109]], dtype=int64)

In [18]:
# Sum of females, males
entropy_x1.values.sum(axis=1)

array([314, 577], dtype=int64)

In [19]:
# Total sum
entropy_x1.values.sum()

891

In [16]:
# As per the formulae above, 
# E(Survived, Gender) = P(Female) * E(Female) + P(Male) * E(Male)
# E(Survived, Gender) = P(Female) * E(81,233) + P(Male) * E(468,109)
p_female, p_male = entropy_x1.values.sum(axis=1)/entropy_x1.values.sum()
p_female, p_male

(0.35241301907968575, 0.6475869809203143)

In [20]:
entropy_x1.values[0]

array([ 81, 233], dtype=int64)

In [23]:
p_fem_surv0, p_fem_surv1 = entropy_x1.values[0] / entropy_x1.values[0].sum()
p_fem_surv0, p_fem_surv1

(0.25796178343949044, 0.7420382165605095)

In [24]:
entropy_fem = -p_fem_surv0 * math.log(p_fem_surv0,2) - p_fem_surv1 * math.log(p_fem_surv1,2)
entropy_fem

0.8236550739295191

In [26]:
p_male_surv0, p_male_surv1 = entropy_x1.values[1] / entropy_x1.values[1].sum()
p_male_surv0, p_male_surv1

(0.8110918544194108, 0.18890814558058924)

In [27]:
entropy_male = -p_male_surv0 * math.log(p_male_surv0,2) - p_male_surv1 * math.log(p_male_surv1,2)
entropy_male

0.6991817891208407

In [28]:
entropy_survived_gender = p_female * entropy_fem + p_male * entropy_male
entropy_survived_gender

0.7430477952150327

E(Survived, Gender) = 0.7430477952150327

We did this for one X variable, we repease this for all X variables, and pick the X variable with the least 
entropy value

<b> Now let's calculate Entropy for E(Survived,Fare)

In [33]:
#E(Survived,Fare)
entropy_x1 = pd.crosstab(dataset['survived'],dataset['fare']).T
entropy_x1

survived,0,1
fare,,
0,464,216
1,85,126


In [37]:
p_fare0, p_fare1 = entropy_x1.values.sum(axis=1)/entropy_x1.values.sum()
p_fare0, p_fare1

(0.7631874298540965, 0.23681257014590348)

In [46]:
entropy_x1.values[0]

array([464, 216], dtype=int64)

In [47]:
entropy_x1.values[1]

array([ 85, 126], dtype=int64)

In [48]:
p_fare0_surv0,p_fare0_surv1 = entropy_x1.values[0]/entropy_x1.values[0].sum()
p_fare1_surv0,p_fare1_surv1 = entropy_x1.values[1]/entropy_x1.values[1].sum()

In [49]:
e_fare0 = -p_fare0_surv0 * (p_fare0_surv0 * math.log(p_fare0_surv0,2)) - p_fare0_surv1 * (p_fare0_surv1 * math.log(p_fare0_surv1,2))
e_fare0

0.42367834531837956

In [50]:
e_fare1 = -p_fare1_surv0 * (p_fare1_surv0 * math.log(p_fare1_surv0,2)) - p_fare1_surv1 * (p_fare1_surv1 * math.log(p_fare1_surv1,2))
e_fare1

0.4781107068404824

In [51]:
e_surv_fare = p_fare0 * e_fare0 + p_fare1 * e_fare1
e_surv_fare

0.4365686127495397

##### Information Gain

IG(X1) = E(Y) - E(Y,X1)
<img src='img/ig-1.png'/> 

The column having the highest information gain can be selected as first level nodes in the tree. For every node of the tree Information Gain calculation is performed (we could see same field appearing again as well)

##### GINI Index

GINI index can be used only when we have to do binary splits(2 child nodes for a parent). With information gain we can have more than binary splits

Higher the GINI index higher the homogeneity

Formulae = p^2 + q^2 

q is nothing but 1-p

<img src='img/gini-1.png'/>

In [56]:
gender = pd.crosstab(dataset['sex'], datset['survived']).values
gender

array([[ 81, 233],
       [468, 109]], dtype=int64)

In [57]:
gender[0]

array([ 81, 233], dtype=int64)

In [55]:
gender[0]/gender[0].sum()

array([0.25796178, 0.74203822])

#### Random Forest Classification

Consider the below dataset,
<img src='img/dt-3.png' />

We divide the input dataset into multiple samples. In the above dataset, you can see the row r6 is part of all 3 samples. Samples can be taken row level or column level(take x1,x2,x3 and y)

We run decision tree on each of the samples and see how the y variable is classified.

We do majority voting and choose the final prediction as the one said by the majority (for this reason we usually go for odd number of occurrences so that a clear majority comes out)

This technique is called Random Forest. Random Forest is a <b> Bagging </b> technique

#### Bootstrapping
To understand bootstrap, suppose it were possible to draw repeated samples (of the same size) from the population of interest, a large number of times. Then, one would get a fairly good idea about the sampling distribution of a particular statistic from the collection of its values arising from these repeated samples. The idea behind bootstrap is to use the data of a sample study at hand as a “surrogate population”, for the purpose of approximating the sampling distribution of a statistic; i.e. to resample (with replacement) from the sample data at hand and create a large number of “phantom samples” known as bootstrap samples.
In other words, We randomly sample with replacement from the n known observations. We then call this a bootstrap sample. Since we allow for replacement, this bootstrap sample most likely not identical to our initial sample. Some data points may be duplicated, and others data points from the initial may be omitted in a bootstrap sample.
An Example:
The following numerical example will help to demonstrate how the process works. If we begin with the sample 2, 4, 5, 6, 6, then all of the following are possible bootstrap samples:

    2 ,5, 5, 6, 6
    4, 5, 6, 6, 6
    2, 2, 4, 5, 5
    2, 2, 2, 4, 6
    2, 2, 2, 2, 2
    4, 6, 6, 6, 6

#### Bagging Classification

<img src='img/bag-boost.png' /> 

In Bagging we run multiple algorithms parallely and then make a decision from the outputs.

In Boosting, we run multiple algorithms sequentially. Output of first algorithm will be fed as input to the next and so on.

Bagging is an approach to ensemble learning that is based on bootstrapping. In short, given a training set, we produce multiple different training sets (called bootstrap samples), by sampling with replacement from the original dataset. Then, for each bootstrap sample, we build a model. The results in an ensemble of models, where each model votes with the equal weight. Typically, the goal of this procedure is to reduce the variance of the model of interest (e.g. decision trees).

<b> Advantages of Bagging techniques includes </b>

Improving the stability and accuracy of machine learning algorithms used in statistical classification and regression. 

It also reduces variance and helps to avoid overfitting. 

Although it is usually applied to decision tree methods, it can be used with any type of method.

## Recording

https://drive.google.com/drive/folders/14wmJ6qDqLQb63PMVdaGf1QqppjWeZcD_